In [1]:
!pip install numpy
!pip install pandas

In [2]:
!pip install scikit-learn

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv
/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/submission.csv


In [4]:
import pandas as pd

file_path = '/kaggle/input/predicta-1-0-predict-the-unpredictable-part-2/daily_data.csv'

data_df = pd.read_csv(file_path)

In [5]:
#shape
data_df.shape

(2893, 17)

In [6]:
data_df.isnull().sum()

day_id                         0
city_id                        0
temperature_celsius            0
condition_text              2414
wind_kph                       0
wind_degree                    0
pressure_mb                    0
precip_mm                      0
humidity                       0
cloud                          0
feels_like_celsius             0
visibility_km                  0
uv_index                       0
gust_kph                       0
air_quality_us-epa-index       0
sunrise                        0
sunset                         0
dtype: int64

In [7]:
data_df.drop(["sunrise","sunset"],inplace=True,axis=1)

In [8]:
data_df.shape

(2893, 15)

In [9]:
import pandas as pd

# Assuming your data is in a DataFrame called data_df
# Separate the non-null records
non_null_df = data_df[data_df['condition_text'].notnull()]

# Separate the null records
null_df = data_df[data_df['condition_text'].isnull()]

# Displaying the shapes of the new DataFrames to confirm the separation
print("Non-null DataFrame shape:", non_null_df.shape)
print("Null DataFrame shape:", null_df.shape)


Non-null DataFrame shape: (479, 15)
Null DataFrame shape: (2414, 15)


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif


In [11]:
# Assuming non_null_df is already defined
# Drop rows with missing values in the target column 'condition_text'
non_null_df = non_null_df.dropna(subset=['condition_text'])

# Encode the target variable 'condition_text'
label_encoder_target = LabelEncoder()
non_null_df['condition_text'] = label_encoder_target.fit_transform(non_null_df['condition_text'])

# Encode the 'city_id' and 'day_id' columns
label_encoder_city = LabelEncoder()
non_null_df['city_id'] = label_encoder_city.fit_transform(non_null_df['city_id'])

label_encoder_day = LabelEncoder()
non_null_df['day_id'] = label_encoder_day.fit_transform(non_null_df['day_id'])

# Define feature columns and target column
features = ['day_id', 'city_id', 'temperature_celsius', 'wind_kph', 'wind_degree', 'pressure_mb',
            'precip_mm', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km',
            'uv_index', 'gust_kph', 'air_quality_us-epa-index']
target = 'condition_text'

# Split data into features and target
X = non_null_df[features]
y = non_null_df[target]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [12]:
# Select the best features using SelectKBest
selector = SelectKBest(score_func=f_classif, k='all')
X_selected = selector.fit_transform(X_scaled, y)

# Get the scores for each feature
feature_scores = selector.scores_
feature_names = np.array(features)
selected_features = feature_names[np.argsort(feature_scores)[-10:]]  # Select top 10 features
X_selected = non_null_df[selected_features]
X_selected_scaled = scaler.fit_transform(X_selected)


In [13]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected_scaled, y, test_size=0.2, random_state=42)

# Define the SVM model and parameters for GridSearchCV
svm_model = SVC()
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto'],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(svm_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best Parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Accuracy on test set: 0.6145833333333334
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98        22
           1       0.75      0.50      0.60        12
           2       0.35      0.58      0.44        12
           3       0.33      0.20      0.25         5
           4       0.75      0.75      0.75         8
           5       0.00      0.00      0.00         4
           6       0.53      0.65      0.59        26
           7       0.00      0.00      0.00         5
           8       0.00      0.00      0.00         2

    accuracy                           0.61        96
   macro avg       0.41      0.41      0.40        96
weighted avg       0.58      0.61      0.59        96



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

# Assuming non_null_df is already defined
# Drop rows with missing values in the target column 'condition_text'
non_null_df = non_null_df.dropna(subset=['condition_text'])

# Encode the target variable 'condition_text'
label_encoder_target = LabelEncoder()
non_null_df['condition_text'] = label_encoder_target.fit_transform(non_null_df['condition_text'])

# Encode the 'city_id' and 'day_id' columns
label_encoder_city = LabelEncoder()
non_null_df['city_id'] = label_encoder_city.fit_transform(non_null_df['city_id'])

label_encoder_day = LabelEncoder()
non_null_df['day_id'] = label_encoder_day.fit_transform(non_null_df['day_id'])

# Define feature columns and target column
features = ['day_id', 'city_id', 'temperature_celsius', 'wind_kph', 'wind_degree', 'pressure_mb',
            'precip_mm', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km',
            'uv_index', 'gust_kph', 'air_quality_us-epa-index']
target = 'condition_text'

# Split data into features and target
X = non_null_df[features]
y = non_null_df[target]

# Use a pipeline to streamline the scaling and modeling process
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif)),
    ('svm', SVC())
])

# Define an extended range of parameters for GridSearchCV
param_grid = {
    'selector__k': [5, 10, 'all'],
    'svm__C': [0.1, 1, 10, 100, 1000],
    'svm__gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'],
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X, y)

# Get the best model
best_model = grid_search.best_estimator_

# Split the data into training and test sets for final evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best Parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.001, svm__kernel=linear; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.001, svm__kernel=linear; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.001, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.001, svm__kernel=rbf; total time=   0.1s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.001, svm__kernel=sigmoid; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.01, svm__kernel=linear; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.01, svm__kernel=linear; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.01, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.01, svm__kernel=rbf; total time=   0.0s
[CV] END selector__k=5, svm__C=0.1, svm__gamma=0.01, svm__kernel=sigmoid; total time=   0.0s
[CV] END selector__

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Random Forest
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif)),
    ('rf', RandomForestClassifier())
])

rf_param_grid = {
    'selector__k': [5, 10, 'all'],
    'rf__n_estimators': [100, 200, 500],
    'rf__max_depth': [None, 10, 20, 30]
}

rf_grid_search = GridSearchCV(rf_pipeline, rf_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
rf_grid_search.fit(X, y)

best_rf_model = rf_grid_search.best_estimator_
best_rf_model.fit(X_train, y_train)
y_rf_pred = best_rf_model.predict(X_test)

print("Random Forest Best Parameters:", rf_grid_search.best_params_)
print("Random Forest Accuracy on test set:", accuracy_score(y_test, y_rf_pred))
print("Random Forest Classification Report:\n", classification_report(y_test, y_rf_pred))

# Gradient Boosting
gb_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(score_func=f_classif)),
    ('gb', GradientBoostingClassifier())
])

gb_param_grid = {
    'selector__k': [5, 10, 'all'],
    'gb__n_estimators': [100, 200, 500],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'gb__max_depth': [3, 4, 5]
}

gb_grid_search = GridSearchCV(gb_pipeline, gb_param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
gb_grid_search.fit(X, y)

best_gb_model = gb_grid_search.best_estimator_
best_gb_model.fit(X_train, y_train)
y_gb_pred = best_gb_model.predict(X_test)

print("Gradient Boosting Best Parameters:", gb_grid_search.best_params_)
print("Gradient Boosting Accuracy on test set:", accuracy_score(y_test, y_gb_pred))
print("Gradient Boosting Classification Report:\n", classification_report(y_test, y_gb_pred))


Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] END selector__k=all, svm__C=10, svm__gamma=0.1, svm__kernel=linear; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=0.1, svm__kernel=linear; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=0.1, svm__kernel=linear; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=1, svm__kernel=rbf; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=1, svm__kernel=rbf; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=1, svm__kernel=rbf; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=1, svm__kernel=sigmoid; total time=   0.0s
[CV] END selector__k=all, svm__C=10, svm__gamma=scale, svm__kernel=linear; total time=   0.1s
[CV] END selector__k=all, svm__C=10, svm__gamma=scale, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C=10, svm__gamma=scale, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV] END selector__k=all, svm__C=1000, svm__gamma=0.001, svm__kernel=sigmoid; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.001, svm__kernel=sigmoid; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.001, svm__kernel=sigmoid; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.01, svm__kernel=linear; total time=   5.9s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.01, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.01, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.01, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.1, svm__kernel=linear; total time=   5.9s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.1, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.1, svm__kernel=poly; total time=   0.0s
[CV] END selector__k=all, svm__C=1000, svm__gamma=0.1, sv

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming data_df['condition_text'] contains the original string labels
label_encoder = LabelEncoder()
data_df['condition_text_encoded'] = label_encoder.fit_transform(data_df['condition_text'])

# Extract day_id to keep it for the submission file
day_ids = data_df['day_id']

# Drop the target and identifier columns
X_data_df = data_df.drop(['condition_text', 'condition_text_encoded', 'day_id'], axis=1)

# Ensure X_data_df has the same columns as X_train used during training
feature_names = X_train.columns.tolist()

# Encode categorical features
for col in X_data_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_data_df[col] = le.fit_transform(X_data_df[col])

# Add missing columns with default value 0 or appropriate value
for col in feature_names:
    if col not in X_data_df.columns:
        X_data_df[col] = 0

# Reorder columns to match the training data
X_data_df = X_data_df[feature_names]

# Predict with Random Forest
rf_predictions = best_rf_model.predict(X_data_df)

# Decode the predictions back to the original string labels
decoded_rf_predictions = label_encoder.inverse_transform(rf_predictions)

# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'day_id': day_ids,
    'condition_text': decoded_rf_predictions
})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)


In [17]:
data_df.shape

(2893, 16)

In [18]:
data_df.head(10)

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,condition_text_encoded
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,9
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,9
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,3
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,0
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,9
5,D0006,C001,20.0,NaN,3.6,96,1019.0,0.0,88,0,20.0,10.0,1.0,11.2,1,9
6,D0007,C001,21.0,Partly Cloudy,4.0,310,1015.0,0.0,100,50,21.0,10.0,1.0,15.1,2,6
7,D0008,C001,21.0,NaN,20.2,330,1011.0,0.0,53,75,21.0,10.0,1.0,17.3,1,9
8,D0009,C001,23.0,NaN,3.6,63,1013.0,0.0,47,75,24.8,10.0,1.0,20.5,1,9
9,D0010,C001,24.0,NaN,15.1,330,1014.0,0.0,54,75,25.3,10.0,1.0,9.7,1,9


# Model 2

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Assuming non_null_df is already defined
# Drop rows with missing values in the target column 'condition_text'
non_null_df = non_null_df.dropna(subset=['condition_text'])

# Encode the target variable 'condition_text'
label_encoder_target = LabelEncoder()
non_null_df['condition_text'] = label_encoder_target.fit_transform(non_null_df['condition_text'])

# Encode the 'city_id' and 'day_id' columns
label_encoder_city = LabelEncoder()
non_null_df['city_id'] = label_encoder_city.fit_transform(non_null_df['city_id'])

label_encoder_day = LabelEncoder()
non_null_df['day_id'] = label_encoder_day.fit_transform(non_null_df['day_id'])

# Define feature columns and target column
features = ['day_id', 'city_id', 'temperature_celsius', 'wind_kph', 'wind_degree', 'pressure_mb',
            'precip_mm', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km',
            'uv_index', 'gust_kph', 'air_quality_us-epa-index']
target = 'condition_text'

# Split data into features and target
X = non_null_df[features]
y = non_null_df[target]

# Use a pipeline to streamline the scaling and modeling process with SMOTE for imbalance handling
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('selector', SelectKBest(score_func=f_classif)),
    ('svm', SVC())
])

# Define an extended range of parameters for GridSearchCV
param_grid = {
    'selector__k': [5, 10, 'all'],
    'svm__C': [0.1, 1, 10, 100, 1000],
    'svm__gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'],
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# Perform GridSearchCV to find the best parameters with stratified k-fold cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=StratifiedKFold(5), scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X, y)

# Get the best model
best_model = grid_search.best_estimator_

# Split the data into training and test sets for final evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
best_model.fit(X_train, y_train)

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best Parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=100, selector__k=all; total time=   3.9s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=200, selector__k=5; total time=   5.9s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=200, selector__k=10; total time=   6.1s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=200, selector__k=all; total time=   7.3s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=200, selector__k=all; total time=   7.3s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=500, selector__k=5; total time=  11.5s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=500, selector__k=10; total time=   9.8s
[CV] END gb__learning_rate=0.1, gb__max_depth=5, gb__n_estimators=500, selector__k=all; total time=  10.4s
[CV] END gb__learning_rate=0.2, gb__max_depth=3, gb__n_estimators=100, selector__k=5; 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE

# Assuming data_df['condition_text'] contains the original string labels
label_encoder_target = LabelEncoder()
data_df['condition_text_encoded'] = label_encoder_target.fit_transform(data_df['condition_text'])

# Extract day_id to keep it for the submission file
day_ids = data_df['day_id']

# Drop the target and identifier columns
X_data_df = data_df.drop(['condition_text', 'condition_text_encoded', 'day_id'], axis=1)

# Ensure X_data_df has the same columns as X_train used during training
feature_names = X_train.columns.tolist()

# Encode categorical features (city_id and any others)
label_encoders = {}
for col in X_data_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_data_df[col] = le.fit_transform(X_data_df[col])
    label_encoders[col] = le

# Add missing columns with default value 0 or appropriate value
for col in feature_names:
    if col not in X_data_df.columns:
        X_data_df[col] = 0

# Reorder columns to match the training data
X_data_df = X_data_df[feature_names]

# Apply scaling and feature selection as done in the pipeline
scaler = StandardScaler()
X_data_df_scaled = scaler.fit_transform(X_data_df)

selector = SelectKBest(score_func=f_classif, k=grid_search.best_params_['selector__k'])
X_data_df_selected = selector.fit_transform(X_data_df_scaled, data_df['condition_text_encoded'])

# Predict with the best SVM model
svm_predictions = best_model.named_steps['svm'].predict(X_data_df_selected)

# Decode the predictions back to the original string labels
decoded_svm_predictions = label_encoder_target.inverse_transform(svm_predictions)

# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'day_id': day_ids,
    'condition_text': decoded_svm_predictions
})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)


/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/conda/lib/python3.10/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


# Model 3

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Assuming non_null_df is already defined and cleaned
# Drop rows with missing values in the target column 'condition_text'
non_null_df = non_null_df.dropna(subset=['condition_text'])

# Encode the target variable 'condition_text'
label_encoder_target = LabelEncoder()
non_null_df['condition_text'] = label_encoder_target.fit_transform(non_null_df['condition_text'])

# Encode the 'city_id' and 'day_id' columns
label_encoder_city = LabelEncoder()
non_null_df['city_id'] = label_encoder_city.fit_transform(non_null_df['city_id'])

label_encoder_day = LabelEncoder()
non_null_df['day_id'] = label_encoder_day.fit_transform(non_null_df['day_id'])

# Define feature columns and target column
features = ['day_id', 'city_id', 'temperature_celsius', 'wind_kph', 'wind_degree', 'pressure_mb',
            'precip_mm', 'humidity', 'cloud', 'feels_like_celsius', 'visibility_km',
            'uv_index', 'gust_kph', 'air_quality_us-epa-index']
target = 'condition_text'

# Split data into features and target
X = non_null_df[features]
y = non_null_df[target]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use a pipeline to streamline the scaling, feature selection, and modeling process with SMOTE for imbalance handling
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('selector', SelectKBest(score_func=f_classif)),
    ('svm', SVC())
])

# Define an extended range of parameters for GridSearchCV
param_grid = {
    'selector__k': [5, 10, 'all'],
    'svm__C': [0.1, 1, 10, 100, 1000],
    'svm__gamma': [0.001, 0.01, 0.1, 1, 'scale', 'auto'],
    'svm__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best Parameters:", grid_search.best_params_)
print("Accuracy on test set:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV] END selector__k=all, svm__C=100, svm__gamma=0.001, svm__kernel=poly; total time=   0.2s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.001, svm__kernel=rbf; total time=   0.1s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.001, svm__kernel=sigmoid; total time=   0.1s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.01, svm__kernel=linear; total time=   1.2s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.01, svm__kernel=linear; total time=   1.7s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.1, svm__kernel=linear; total time=   0.8s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.1, svm__kernel=rbf; total time=   0.1s
[CV] END selector__k=all, svm__C=100, svm__gamma=0.1, svm__kernel=sigmoid; total time=   0.1s
[CV] END selector__k=all, svm__C=100, svm__gamma=1, svm__kernel=linear; total time=   1.2s
[CV] END selector__k=all, svm__C=100, svm__gamma=1, svm__kernel=poly; total time=   0.1s
[CV] END s

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Assuming data_df['condition_text'] contains the original string labels
label_encoder = LabelEncoder()
data_df['condition_text_encoded'] = label_encoder.fit_transform(data_df['condition_text'])

# Extract day_id to keep it for the submission file
day_ids = data_df['day_id']

# Drop the target and identifier columns
X_data_df = data_df.drop(['condition_text', 'condition_text_encoded', 'day_id'], axis=1)

# Ensure X_data_df has the same columns as X_train used during training
feature_names = X_train.columns.tolist()

# Encode categorical features
for col in X_data_df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_data_df[col] = le.fit_transform(X_data_df[col])

# Add missing columns with default value 0 or appropriate value
for col in feature_names:
    if col not in X_data_df.columns:
        X_data_df[col] = 0

# Reorder columns to match the training data
X_data_df = X_data_df[feature_names]

# Predict with Random Forest
rf_predictions = best_rf_model.predict(X_data_df)

# Decode the predictions back to the original string labels
decoded_rf_predictions = label_encoder.inverse_transform(rf_predictions)

# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'day_id': day_ids,
    'condition_text': decoded_rf_predictions
})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)
